In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import json
import os
from datetime import datetime

todo_items = []

title_input = widgets.Text(placeholder='할 일 입력')
progress_input = widgets.IntSlider(min=0, max=100, step=5, value=0)
due_input = widgets.DatePicker(placeholder='마감일 (YYYY-MM-DD)')
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통')

add_btn = widgets.Button(description='추가', button_style='success')
save_btn = widgets.Button(description='저장', button_style='info')
load_btn = widgets.Button(description='불러오기', button_style='warning')

progress_bar = widgets.FloatProgress(value=0, min=0, max=100, bar_style='info')
progress_text = widgets.Label()

list_box = widgets.VBox()


In [ ]:
def update_overall_progress():
    if not todo_items:
        progress_bar.value = 0
        progress_text.value = '(0%)'
        return
    avg = sum(item['progress'] for item in todo_items) / len(todo_items)
    progress_bar.value = avg
    progress_text.value = f"({int(avg)}%)"

def refresh_list():
    list_box.children = []
    for idx, item in enumerate(todo_items, 1):
        box_style = 'info' if item['progress'] == 100 else ''
        task_title = f"# {idx}. {item['title']}"
        task_progress = widgets.FloatProgress(value=item['progress'], min=0, max=100, style={'bar_color': 'gray'})

        slider = widgets.IntSlider(min=0, max=100, step=5, value=item['progress'])
        progress_label = widgets.Label(f"{item['progress']}%")

        def update_progress(change, i=item, s=slider, l=progress_label, p=task_progress):
            i['progress'] = change['new']
            l.value = f"{change['new']}%"
            p.value = change['new']
            update_overall_progress()

        slider.observe(update_progress, names='value')

        date_label = widgets.Label(f"마감일 {item['due']}")
        priority_label = widgets.Label(f"중요도 {item['priority']}")
        deadline = widgets.HBox([date_label, priority_label])

        done_btn = widgets.Button(description='완료✅', button_style='', layout=widgets.Layout(width='50%'))
        delete_btn = widgets.Button(description='삭제❌', button_style='', layout=widgets.Layout(width='50%'))

        def on_done(b, i=item):
            i['progress'] = 100
            refresh_list()
            update_overall_progress()

        def on_delete(b, i=item):
            todo_items.remove(i)
            refresh_list()
            update_overall_progress()

        done_btn.on_click(on_done)
        delete_btn.on_click(on_delete)

        controls = widgets.HBox([done_btn, delete_btn])
        entry = widgets.VBox([
            widgets.HTML(f"<b>{task_title}</b>"),
            task_progress,
            widgets.HBox([slider, progress_label]),
            deadline,
            controls
        ])
        list_box.children += (entry,)


In [ ]:
def add_task(b):
    if not title_input.value.strip():
        return
    todo_items.append({
        'title': title_input.value.strip(),
        'progress': progress_input.value,
        'due': due_input.value.strftime('%Y-%m-%d') if due_input.value else '',
        'priority': priority_input.value
    })
    title_input.value = ''
    progress_input.value = 0
    due_input.value = None
    priority_input.value = '보통'
    refresh_list()
    update_overall_progress()

def save_data(b):
    with open('todo.json', 'w') as f:
        json.dump(todo_items, f)

def load_data(b):
    if not os.path.exists('todo.json'):
        return
    with open('todo.json', 'r') as f:
        data = json.load(f)
        todo_items.clear()
        todo_items.extend(data)
        refresh_list()
        update_overall_progress()

add_btn.on_click(add_task)
save_btn.on_click(save_data)
load_btn.on_click(load_data)


In [ ]:
display(widgets.HTML("<h3 style='color:black;'>📁 DAVID mk40 - 새 시작</h3>"))

display(widgets.VBox([
    title_input,
    progress_input,
    widgets.HBox([due_input, priority_input]),
    add_btn,
    widgets.HBox([save_btn, load_btn]),
    widgets.HTML("<b>전체 진행률</b>"),
    widgets.HBox([progress_bar, progress_text]),
    list_box
]))

refresh_list()
update_overall_progress()
